# Vuegen Basic Case Study - Predefined Directory

[![Open In Colab][colab_badge]][colab_link]

This notebook is a basic demo of the Vuegen Python library. This sofwtare automates the creation of reports based on a directory with plots, dataframes, and other files in different formats. A YAML configuration file is generated from the directory to define the structure of the report. Users can customize the report by modifying the configuration file, or they can create their own configuration file instead of passing a directory as input. 

The configuration file specifies the structure of the report, including sections, subsections, and various components such as plots, dataframes, markdown, html, and API calls. Reports can be generated in various formats, including documents (PDF, HTML, DOCX, ODT), presentations (PPTX, Reveal.js), notebooks (Jupyter) or Streamlit web applications.

An overview of the VueGen workflow is shown in the figure below:

![Vuegen graphical abstarct][abstractfig_vuegen]

This introductory case study familiarizes users with the tool, its report types, file formats, and other features. In this example, a directory with plots, dataframes, Markdown, and HTML components is provided. An advanced example can be found [here][advanced_notebook].

## Notebook structure

First, we will set up the work environment by installing the necessary packages and importing the required libraries. Next, we will create various reports using the example directory. Finally, we will extend the report by modifying the configuration file to include additional components.

0. [Work environment setup](#0-work-environment-setup)
1. [Report generation](#1-report-generation)
2. [Extending the report](#2-extending-the-report)

## Credits and Contributors
- This notebook was created by Sebastián Ayala-Ruano under the supervision of Henry Webel and Alberto Santos, head of the [Multiomics Network Analytics Group (MoNA)][Mona] at the [Novo Nordisk Foundation Center for Biosustainability (DTU Biosustain)][Biosustain].
- You can find more details about the project in this [GitHub repository][githubrepo].

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_link]: https://colab.research.google.com/github/Multiomics-Analytics-Group/vuegen/blob/main/docs/vuegen_basic_case_study.ipynb
[abstractfig_vuegen]: https://raw.githubusercontent.com/Multiomics-Analytics-Group/vuegen/main/docs/images/vuegen_graph_abstract.png
[emp_repo]: https://github.com/biocore/emp/tree/master
[emp_paper]: https://www.nature.com/articles/nature24621
[Mona]: https://multiomics-analytics-group.github.io/
[Biosustain]: https://www.biosustain.dtu.dk/
[githubrepo]: https://github.com/Multiomics-Analytics-Group/vuegen
[advanced_notebook]: https://github.com/Multiomics-Analytics-Group/vuegen/blob/main/docs/vuegen_case_study_earth_microbiome.ipynb


## 0. Work environment setup

### 0.1. Installing libraries and creating global variables for platform and working directory

To run this notebook locally, you should create a virtual environment with the required libraries. If you are running this notebook on Google Colab, everything should be set.

In [ ]:
# Vuegen library
%pip install vuegen

In [ ]:
import os

IN_COLAB = "COLAB_GPU" in os.environ

In [ ]:
# Set working directory
if IN_COLAB:
    # Clone the repository in Colab
    !git clone --depth=1 https://github.com/Multiomics-Analytics-Group/vuegen.git
    base_output_dir = "vuegen/docs/example_data/Basic_example_vuegen_demo_notebook/"
else:
    # Output directory for local execution
    base_output_dir = "./example_data/Basic_example_vuegen_demo_notebook/"

In [ ]:
# Optional library to launch a streamlit app from colab
if IN_COLAB:
    !npm install localtunnel

### 0.2. Importing libraries

In [ ]:
# Imports
import os
import yaml
from vuegen import report_generator
from vuegen.utils import get_logger, load_yaml_config

if IN_COLAB:
    import urllib

## 1. Report generation
To generate different report types, just modify the report_type variable. The available types are: 
* streamlit
* html
* pdf
* docx
* odt
* revealjs
* pptx
* jupyter

### 1.1. Streamlit report

To launch the Streamlit web application from Colab, open the generated URL and copy the localtunnel entry point IP into the corresponding field on the opened page. Once submited, you will be redirected to your Streamlit web application.

In [ ]:
# Generate the report
report_type = "streamlit"
report_dir, config_path = report_generator.get_report(
    dir_path=base_output_dir, report_type=report_type, logger=None
)
print(f"\nReport generated in {report_dir}")
print(f"\nConfig file generated in {config_path}")

In [ ]:
run_streamlit = False
# run_streamlit = True # uncomment line to run the streamlit report
# Launch the Streamlit report depneding on the platform
if not IN_COLAB and run_streamlit:
    !streamlit run streamlit_report/sections/report_manager.py
elif run_streamlit:
    # see: https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399
    print(
        "Password/Enpoint IP for localtunnel is:",
        urllib.request.urlopen("https://ipv4.icanhazip.com")
        .read()
        .decode("utf8")
        .strip("\n"),
    )
    # Run the Streamlit app in the background
    !streamlit run streamlit_report/sections/report_manager.py --server.address=localhost &>/content/logs.txt &
    # Expose the Streamlit app on port 8501
    !npx localtunnel --port 8501 --subdomain vuegen-demo
else:
    print("Streamlit report not executed, set run_streamlit to True to run the report")

### 1.2. HTML and other report types

In [ ]:
# Generate the report
report_type = "html"
report_dir, config_path = report_generator.get_report(
    dir_path=base_output_dir, report_type=report_type, logger=None
)
print(f"Report generated at: {report_dir}")

## 2. Extending the report
Now, we will extend the report by modifying the configuration file to include a logo and graphica abstract in the main page, a descritption for a section and a subsection, and a new plot from a url. We are modifying this file from the notebook, but it is also possible to do it directly in the configuration file with a text editor.

### 2.1. Adding a logo and graphical abstract

In [ ]:
vuegen_logo_path = "https://raw.githubusercontent.com/Multiomics-Analytics-Group/vuegen/main/docs/images/vuegen_logo.svg"

# Load the YAML file
print(
    f"Loading the YAML config file from: {config_path}"
)  # generated based on directory path above
config = load_yaml_config(config_path)

# Update the logo and graphical abstract with the URL
config["report"].update(
    {"logo": vuegen_logo_path, "graphical_abstract": vuegen_logo_path}
)

### 2.2. Including a description for a section and a subsection

In [ ]:
# Update the description for the EDA section
for section in config["sections"]:
    if section["title"] == "Plots":
        section["description"] = "This section contains example plots"

# Update the description for the alpha diversity subsection from the Metagenomics section
for section in config["sections"]:
    if section["title"] == "Dataframes":
        for subsection in section["subsections"]:
            if subsection["title"] == "All Formats":
                subsection["description"] = (
                    "This subsection contains example dataframes."
                )

### 2.3. Adding a new plot from a url

In [ ]:
# Define new plot with a URL as the file path
vuegen_abst_fig = {
    "title": "Graphical overview of VueGen’s workflow and components",
    "file_path": "https://raw.githubusercontent.com/Multiomics-Analytics-Group/vuegen/main/docs/images/vuegen_graph_abstract.png",
    "description": "",
    "caption": "The diagram illustrates the processing pipeline of VueGen, starting from either a directory or a YAML configuration file. Reports consist of hierarchical sections and subsections, each containing various components such as plots, dataframes, Markdown, HTML, and data retrieved via API calls.",
    "component_type": "plot",
    "plot_type": "static",
}

# Add the plot to the Sample Provenance subsection in the EDA section
for section in config["sections"]:
    if section["title"] == "Plots":
        for subsection in section["subsections"]:
            if subsection["title"] == "Static Plots":
                subsection["components"].append(vuegen_abst_fig)

# Save the modified YAML file
with open(config_path, "w") as file:
    yaml.dump(config, file, default_flow_style=False)

### 2.5. Stremlit report with the extended configuration file

To launch the Streamlit web application from Colab, open the generated URL and copy the localtunnel entry point IP into the corresponding field on the opened page. Once submited, you will be redirected to your Streamlit web application.

In [ ]:
# Test the changes by generarating the report from the modified YAML file
report_type = "streamlit"
_ = report_generator.get_report(
    config_path=config_path, report_type=report_type, logger=None
)

In [ ]:
run_streamlit = False
# run_streamlit = True # uncomment line to run the streamlit report
# Launch the Streamlit report depneding on the platform
if not IN_COLAB and run_streamlit:
    !streamlit run streamlit_report/sections/report_manager.py
elif run_streamlit:
    # see: https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399
    print(
        "Password/Enpoint IP for localtunnel is:",
        urllib.request.urlopen("https://ipv4.icanhazip.com")
        .read()
        .decode("utf8")
        .strip("\n"),
    )
    # Run the Streamlit app in the background
    !streamlit run streamlit_report/sections/report_manager.py --server.address=localhost &>/content/logs.txt &
    # Expose the Streamlit app on port 8501
    !npx localtunnel --port 8501 --subdomain vuegen-demo
else:
    print("Streamlit report not executed, set run_streamlit to True to run the report")

### 2.6. HTML and other report types with the extended configuration file

In [ ]:
# Test the changes by generarating the report from the modified YAML file
report_type = "html"
_ = report_generator.get_report(
    config_path=config_path, report_type=report_type, logger=None
)